In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import json
from tqdm import tqdm

# Finam

##  Получение Российских акций

In [ ]:
url = "https://www.finam.ru/quotes/stocks/russia/"
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
news_section = driver.find_element(By.ID, 'finfin-local-plugin-quote-table-table-more-container')
news_section.click()

In [ ]:
table = driver.find_element(By.ID, 'finfin-local-plugin-quote-table-table-table')

In [ ]:
list_of_instr = table.find_elements(By.TAG_NAME, 'tr')

In [ ]:
len(list_of_instr)

In [ ]:
list_id = []
list_link = []
list_name = []

for i in range(1,301):
    list_id.append(list_of_instr[i].get_attribute('data-quoteid'))
    list_link.append(list_of_instr[i].find_element(By.TAG_NAME, 'a').get_attribute('href'))
    list_name.append(list_of_instr[i].find_element(By.TAG_NAME, 'td').text)


In [ ]:
data_stock = {
    'em': list_id,
    'name': list_name,
    'link': list_link
}

df_stock = pd.DataFrame(data_stock)
df_stock.to_parquet('finam_ru_stock_300.parquet')
df_stock

In [ ]:
driver.quit()

## Получение тех информации по акциям

In [ ]:
driver = webdriver.Chrome()

Перебираем все акции  и получаем их тикеры и инструменты прожажи

In [ ]:
list_ticker = []
list_market_id = []
list_market_name = []

In [ ]:
for url_stock in tqdm(list_link):
    driver.get(url_stock) # переход на страницу акции
    
    # поиск экспорта
    export = driver.find_element(By.ID, 'finfin-local-plugin-quote-item-item-tabs-content-tech')
    
    try:
        export_page = export.find_elements(By.TAG_NAME, 'a')[1].get_attribute('href')
    except:
        list_ticker.append(None)
        list_market_id.append(None)
        list_market_name.append(None)
        continue
    
    driver.get(export_page) # переход на страницу экспорта 

    # поиск данных акции
    info = driver.find_element(By.ID, 'content-block')
    script_tag = info.find_element(By.TAG_NAME, 'script')
    script = script_tag.get_attribute('text')
    
    # отчистка
    string_info = script.split("Finam.")[4].replace('\n', '')
    test_string = string_info.replace('IssuerProfile.Main.issue = ', '').replace(', }', '}').replace(',}', '}').replace('&quot', '')
    res = json.loads(test_string[:-1]) # получение словаря 

    list_ticker.append(res['quote']['code'])
    list_market_id.append(res['quote']['market']['id'])
    list_market_name.append(res['quote']['market']['title'])

In [ ]:
data_stock = {
    'em': list_id,
    '_company_name': list_name,
    'link': list_link,
    'ticker': list_ticker,
    'market_id': list_market_id,
    'market_name': list_market_name
}

df_stock = pd.DataFrame(data_stock)
df_stock.to_parquet('finam_ru_stock_300.parquet')
df_stock.dropna()

In [ ]:
driver.quit()

## Получение тех информации по тикеру

In [120]:
def get_tech_info(ticker):
    url =  f'https://www.finam.ru/quote/moex/{ticker}/' 
    driver = webdriver.Chrome()
    driver.get(url) # переход на страницу акции
    
    # поиск экспорта
    export = driver.find_element(By.ID, 'finfin-local-plugin-quote-item-item-tabs-content-tech')
    
    try:
        export_page = export.find_elements(By.TAG_NAME, 'a')[1].get_attribute('href')
    except:
        # list_ticker.append(None)
        # list_market_id.append(None)
        # list_market_name.append(None)
        return False 
    
    driver.get(export_page) # переход на страницу экспорта 

    # поиск данных акции
    info = driver.find_element(By.ID, 'content-block')
    script_tag = info.find_element(By.TAG_NAME, 'script')
    script = script_tag.get_attribute('text')
    
    driver.quit()
    
    # отчистка
    string_info = script.split("Finam.")[4].replace('\n', '')
    test_string = string_info.replace('IssuerProfile.Main.issue = ', '').replace(', }', '}').replace(',}', '}').replace('&quot', '')
    res = json.loads(test_string[:-1]) # получение словаря 
    
    return res['quote']['id'], res['quote']['code'], res['quote']['market']['id'],  res['quote']['market']['title']

In [121]:
get_tech_info('YNDX')

(388383, 'YNDX', 1, 'МосБиржа акции')

## Получение списка бирж 

In [ ]:
test_string = script.split(";")[2].replace('\n', '').replace('Finam.IssuerProfile.Main.setMarkets([', '').replace(', }', '}').replace(',}', '}').replace('])', '')

In [ ]:
list_shops = test_string.split('}, ')

for i in range(len(list_shops) - 1):
    list_shops[i] = (list_shops[i] + '}')

for i in range(len(list_shops)):
    list_shops[i] = list_shops[i].replace('\'', '\"')

    dict_split = list_shops[i].split(' ')
    dict_split[1] = ' \"' + dict_split[1][:-1] + '\"' + ': '
    dict_split[3] = ' \"' + dict_split[3][:-1] + '\"' + ': '
    list_shops[i] = json.loads(''.join(dict_split))


In [ ]:
list_value = []
list_title = []

for i in list_shops:
    list_value.append(i['value'])
    list_title.append(i['title'])

data_shops = {
    'value': list_value,
    'title': list_title
    }

shops = pd.DataFrame(data_shops)
shops

In [ ]:
shops.to_parquet('ru_market.parquet')

## Получение чистых тикеров 

In [145]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [140]:
df = pd.read_parquet('finam_ru_stock_300.parquet')
df = df[df["market_id"] == 1]

In [168]:
# df[df["len_mask"] > 4]['link']

In [144]:
df["ticker"] = df["ticker"].apply(lambda x: x if len(x) <= 5 else x.split('.')[1].split(":")[0])

In [147]:
df["len_mask"] = df["ticker"].apply(lambda x: len(x))

In [167]:
driver = webdriver.Chrome()

# df[df["len_mask"] > 4]['link']

for i in df[df["len_mask"] > 4]['link']:
    url = i[:-2]
    print(url)
    driver.get(url)
    try:
        driver.find_element(By.ID, 'finfin-local-plugin-quote-item-item-tabs-content-tech')
    except:
        print('404')

404
https://www.finam.ru/quote/moex/trnf - 20
https://www.finam.ru/quote/moex/himc - 158
https://www.finam.ru/quote/moex/omzz - 190
https://www.finam.ru/quote/moex/krko - 206 
https://www.finam.ru/quote/moex/bisv - 209
https://www.finam.ru/quote/moex/veon-r - 293 -- НЕ ПРЕФ

In [202]:
only_pref_list = ['trnf', 'himc', 'omzz', 'krko', 'bisv']

In [175]:
df['pref'] = df["ticker"].apply(lambda x: str(x) in list(df[df["len_mask"] > 4]['ticker']))
df['pref'][293] = False # не преф, просто длинный тикер

C:\Users\pledushka\AppData\Local\Temp\ipykernel_24276\4232253078.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pref'][293] = False # не преф, просто длинный тикер


In [187]:
pref_list = list(df[df['pref']]['ticker'])

In [205]:
# получаю список обычных акций из префов 
tickers_from_pref_list = [] 

for i in pref_list:
    if i[:-1].lower() not in only_pref_list:
        tickers_from_pref_list.append(i[:-1])

len(tickers_from_pref_list)

46

In [212]:
count = 0
for i in tickers_from_pref_list:
    if i in list(df['ticker']):
        count += 1
count

46

## Сопоставление КОМПАНИЯ - ТИКЕР

In [5]:
def get_ticker_by_company(company_name):
    url = "https://www.finam.ru/quotes/indices/world/"
    driver = webdriver.Chrome()
    driver.get(url)

    search = driver.find_element(By.ID, 'infinity-ui-left-header-menu-search-trigger').click()
    search_input = driver.find_element(By.ID, 'infinity-ui-left-header-menu-search').find_element(By.TAG_NAME, 'input')
    search_input.send_keys(company_name)
    
    search_list = driver.find_element(By.ID, 'infinity-ui-left-header-menu-search-result')
    
    # driver.implicitly_wait(1) # seconds

    try:
        element = WebDriverWait(search_list, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'a'))
        )
    except:
        # driver.quit()
        return None
    
    tickers_a = search_list.find_elements(By.TAG_NAME, 'a')
    for el in tickers_a: # московская биржа может быть не первой в списке. пример тинькофф - TCS Group
        stock_ticker = el.get_attribute('data-chpurl').split('/') # пара (биржа/тикер)

        if stock_ticker[0] == 'moex':
            return stock_ticker[1]

    return None

In [8]:
a = get_ticker_by_company('Рус Гидро')
print(a)

hydr


## Получение обычных акций из префов

По итогу выяснилось, что все обычные акции, имеющие префы, также есть в списке. Задача свелась в удаление выбросов

In [130]:
df = pd.read_parquet('finam_ru_stock_300.parquet')

In [124]:
df.head(1)

,em,_company_name,link,ticker,market_id,market_name
0,29,Аэрофлот,https://www.finam.ru/quote/moex/aflt/,AFLT,1.0,МосБиржа акции


In [131]:
df = df[df['market_id'] == 1]

In [218]:
df = df.reset_index()
df = df.drop('index', axis=1)

In [220]:
df.tail(1)

,em,_company_name,link,ticker,market_id,market_name,pref
252,4271197,Новошип ао,https://www.finam.ru/quote/moex/nomp/,NOMP,1.0,МосБиржа акции,False


In [319]:
df.to_parquet('clear_tikers.parquet')

# df = pd.read_parquet('clear_tikers.parquet')

In [302]:
df.shape

(253, 7)

In [257]:
import asyncio
import aiohttp
import aiomoex
import pandas as pd


async def stock_history(ticker):
    """
    История изменения цены бумаги
    """
    async with aiohttp.ClientSession() as session:
        data = await aiomoex.get_board_history(session,  ticker)
        df = pd.DataFrame(data)
        print(df)

async def stock_candles(ticker, interval=24, start = None, end = None):
    """
    Свечки по бумаге.
    Размер свечки - целое число 1 (1 минута), 10 (10 минут), 60 (1 час), 
    24 (1 день), 7 (1 неделя), 31 (1 месяц) или 4 (1 квартал). По умолчанию дневные данные.
    """
    async with aiohttp.ClientSession() as session:
        data = await aiomoex.get_board_candles(session,  ticker, interval=interval, start=start, end=end)
        df = pd.DataFrame(data)
        print(df)

async def stock_list():
    """
    Список торгуемых бумаг.
    """
    request_url = "https://iss.moex.com/iss/engines/stock/" "markets/shares/boards/TQBR/securities.json"
    arguments = {"securities.columns": ("SECID," "REGNUMBER," "LOTSIZE," "SHORTNAME")}

    async with aiohttp.ClientSession() as session:
        iss = aiomoex.ISSClient(session, request_url, arguments)
        data = await iss.get()
        df = pd.DataFrame(data["securities"])
        df.set_index("SECID", inplace=True)
        
        # print(df)
        return df

async def correlation():
    """
    Корреляция одного тикера к другому.
    """
    #TODO: load all history
    request_url = "https://iss.moex.com/iss/statistics/engines/stock/markets/shares/correlations.json"
    arguments = {}
    
    async with aiohttp.ClientSession() as session:
        iss = aiomoex.ISSClient(session, request_url)
        data = await iss.get()
        df = pd.DataFrame(data["coefficients"])
        print(df)

async def netflow():
    """
    Аггрегированный объем.
    """
    request_url = "https://iss.moex.com/iss/analyticalproducts/netflow2/securities.json"
    arguments = { "date": "2013-07-12" }
    async with aiohttp.ClientSession() as session:
        iss = aiomoex.ISSClient(session, request_url, arguments)
        data = await iss.get()
        df = pd.DataFrame(data["netflow2"])
        print(df)



        
df_moex = await stock_list()

In [264]:
df_moex = df_moex.reset_index()

In [267]:
t_moex = list(df_moex['SECID'])
t_finam = list(df['ticker'])

In [268]:
len(t_finam)

253

In [269]:
len(t_moex)

247

In [274]:
count = 0

trash = []

for i in t_finam:
    if i not in t_moex:
        trash.append(i)

In [289]:
trash

['UWGN',
 'LPSB',
 'SLAV',
 'NOMPP',
 'ORUP',
 'NTZL',
 'ZILLP',
 'OBNEP',
 'GRNT',
 'KZIZ',
 'OBNE',
 'NOMP']

In [313]:
df = pd.read_parquet('clear_tikers.parquet')

In [314]:
df.shape

(253, 7)

In [315]:
df = df.query(f"ticker not in {trash}")

In [317]:
df = df.reset_index()
del df['index']

In [318]:
df

,em,_company_name,link,ticker,market_id,market_name,pref
0,29,Аэрофлот,https://www.finam.ru/quote/moex/aflt/,AFLT,1.0,МосБиржа акции,False
1,19043,ВТБ ао,https://www.finam.ru/quote/moex/vtbr/,VTBR,1.0,МосБиржа акции,False
2,16842,ГАЗПРОМ ао,https://www.finam.ru/quote/moex/gazp/,GAZP,1.0,МосБиржа акции,False
3,795,ГМКНорНик,https://www.finam.ru/quote/moex/gmkn/,GMKN,1.0,МосБиржа акции,False
4,8,ЛУКОЙЛ,https://www.finam.ru/quote/moex/lkoh/,LKOH,1.0,МосБиржа акции,False
5,15523,МТС-ао,https://www.finam.ru/quote/moex/mtss/,MTSS,1.0,МосБиржа акции,False
6,17086,Магнит ао,https://www.finam.ru/quote/moex/mgnt/,MGNT,1.0,МосБиржа акции,False
7,152798,МосБиржа,https://www.finam.ru/quote/moex/moex/,MOEX,1.0,МосБиржа акции,False
8,17046,НЛМК ао,https://www.finam.ru/quote/moex/nlmk/,NLMK,1.0,МосБиржа акции,False
9,17273,Роснефть,https://www.finam.ru/quote/moex/rosn/,ROSN,1.0,МосБиржа акции,False
